# Cours de SQL

Ce cours est présenté sous forme de Notebook, je le mettrai à jour après chaque séance. Il vous suffit de sélectionner une base de donnée et de jouer l'ensemble du notebook pour être prêt à aborder une nouvelle séance.

## Rappels

Nous travaillons avec SQL Server 2019, qui utilise la syntaxe Transact-SQL. En cas de problème/blocage, référez-vous à la documentation de Transact-SQL de préférence. D'autres documentations (MySQL, Postgres, Oracle, PL/SQL) pourraient vous aider, mais il existe des différences entre ces différents dialectes.

## Contexte

Nous allons prendre pour contexte une base de donnée de site de e-commerce assez sommaire. Seule une partie de la base va être modélisée, le but étant de présenter uniquement des notions de base de donnée et non de faire un projet complet.

## Révisions

Pour commencer, nous allons créer une base de données et nous positionner dessus pour travailler.

In [1]:
-- On teste l'existence de la base pour éviter de
-- crasher le notebook si elle existe déjà
IF(DB_ID(N'eCommerce') IS NULL)
    CREATE DATABASE [eCommerce];
GO
USE [eCommerce];
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.055

Maintenant que nous avons une base de données, nous allons pouvoir créer une table. Nous allons commencer par la table des produits. Pour le moment, nous produits auront un `Id` ainsi qu'un `Nom`.

Afin de faire des colones avec un auto-incrément, nous allons utiliser la propriété `Identity`. Nous allons également positionner une clé primaire sur notre colone `Id`.

In [2]:
IF NOT EXISTS (
    SELECT  [name]
    FROM    [sysobjects]
    WHERE   [name]='Produit_PRD'
            AND [xtype]='U')
    CREATE TABLE [dbo].[Produit_PRD] (
        [Id] BIGINT IDENTITY(1, 1) PRIMARY KEY,
        [Nom] NVARCHAR(255) NOT NULL,
    )
    GO

Commands completed successfully.

Total execution time: 00:00:00.061

Nous allons également ajouter une table qui contiendra des revues sur les différents articles proposés sur le site. Les revues auront une `Id`, clé primaire identité, une clé étrangère vers la table des produits, un titre et un contenu.

In [3]:
IF NOT EXISTS (
    SELECT  [name]
    FROM    [sysobjects]
    WHERE   [name]='Revue_REV'
            AND [xtype]='U')
    CREATE TABLE [dbo].[Revue_REV] (
        [Id] BIGINT IDENTITY(1, 1) PRIMARY KEY,
        [PRD_Id] BIGINT NOT NULL,
        [Titre] NVARCHAR(255) NOT NULL,
        [Contenu] NVARCHAR(MAX) NOT NULL,
        CONSTRAINT fk_PRD_Id_REV
            FOREIGN KEY ([PRD_Id]) REFERENCES [Produit_PRD]([Id])
    )
GO

Commands completed successfully.

Total execution time: 00:00:00.007

### Opérations de base

Maintenant que nous avons des tables, nous allons pouvoir y mettre du contenu, le récupérer, le modifier, le supprimer.

Pour l'insertion, nous ne devons pas saisir de valeur pour les colonnes identités. Ainsi, si je souhaite insérer une ligne dans ma table de produit, je pourrais faire la requête suivante :

In [4]:
/*
INSERT INTO [dbo].[Produit_PRD] ([Nom])
VALUES (N'iPone 11'), (N'One Plus 7')
GO
*/

Commands completed successfully.

Total execution time: 00:00:00.001

On peut récupérer les données avec l'instruction `SELECT`

In [5]:
SELECT
        [Id],
        [Nom]
FROM    [dbo].[Produit_PRD]
GO

(1 row affected)

Total execution time: 00:00:00.101

Id,Nom
1,iPhone 11


On va pouvoir faire des mises à jour à l'aide de la clause `UPDATE`

In [6]:
/*
UPDATE [dbo].[Produit_PRD]
SET [Nom] = N'iPhone 11'
WHERE [Id] = 1
GO
*/

Commands completed successfully.

Total execution time: 00:00:00.001

On peut également supprimer des lignes avec l'instruction `DELETE`

In [7]:
/*
DELETE FROM [dbo].[Produit_PRD] WHERE [Id] = 2
GO
*/

Commands completed successfully.

Total execution time: 00:00:00.001

Nous pouvons modifier la structure des table à postériori, à l'aide de l'instruction `ALTER TABLE`. Nous allons modifier notre table de produits pour ajouter une colone avec le prix, un descriptif et un contenu.

In [8]:
IF NOT EXISTS (
    SELECT [name] 
    FROM   [sys].[columns]
    WHERE  [object_id] = OBJECT_ID(N'[dbo].[Produit_PRD]') 
            AND [name] = 'Prix'
)
    ALTER TABLE [dbo].[Produit_PRD]
    ADD [Prix] DECIMAL(5, 2) NOT NULL,
        [Descriptif] NVARCHAR(255) NULL,
        [Contenu] NVARCHAR(MAX) NULL,
        DEFAULT 0 FOR [Prix]
GO

Commands completed successfully.

Total execution time: 00:00:00.015

### Jointures

Nous pouvons faire des jointures entre le contenu de différentes tables. Il existe différents types de jointures :

![Jointures SQL](https://i.pinimg.com/originals/78/06/27/78062746cee62bdc112aad0582ae49c4.jpg)

Ceci va nous permettre de créer des requêtes complexes.

In [9]:
SELECT
    [PRD].[Id] AS [ProduitId],
    [PRD].[Nom] AS [ProduitNom],
    [REV].[Titre] AS [RevueTitre]
FROM [dbo].[Produit_PRD] AS [PRD]
INNER JOIN [dbo].[Revue_REV] AS [REV]
    ON [REV].[PRD_Id] = [PRD].[Id]
ORDER BY [PRD].[Id] ASC
GO

(0 rows affected)

Total execution time: 00:00:00.008

ProduitId,ProduitNom,RevueTitre


### Vues

Nous pouvons utiliser des vues pour avoir une vision de plusieurs tables aglomérées et mise en cache par le SGBD.

In [10]:
CREATE OR ALTER VIEW [dbo].[VueRevues]
AS
    SELECT
        [PRD].[Id] AS [ProduitId],
        [PRD].[Nom] AS [ProduitNom],
        [REV].[Titre] AS [RevueTitre]
    FROM [dbo].[Produit_PRD] AS [PRD] (NOLOCK)
    INNER JOIN [dbo].[Revue_REV] AS [REV] (NOLOCK)
        ON [REV].[PRD_Id] = [PRD].[Id]
GO

Commands completed successfully.

Total execution time: 00:00:00.007

In [11]:
SELECT [ProduitId], [ProduitNom], [RevueTitre]
FROM [dbo].[VueRevues]
GO

(0 rows affected)

Total execution time: 00:00:00.006

ProduitId,ProduitNom,RevueTitre


## Fonctions

En SQL, nous pouvons créer différents types de fonctions. Nous en étudierons deux :
- Les fonctions table
- Les fonctions scalaires

Les fonctions tables sont des fonctions qui vont nous revoyer des données structurées sous forme d'une table.

In [12]:
CREATE OR ALTER FUNCTION [dbo].[GetRevues](@PrdId BIGINT)
RETURNS TABLE
AS
RETURN
(
    SELECT [Titre], [Contenu]
    FROM [dbo].[Revue_REV] (NOLOCK)
    WHERE [PRD_Id] = @PrdId
)
GO

Commands completed successfully.

Total execution time: 00:00:00.006

In [13]:
SELECT [Titre], [Contenu] FROM [dbo].[GetRevues](1)
GO

(0 rows affected)

Total execution time: 00:00:00.005

Titre,Contenu


Nous avons également les fonctions scalaires, qui vont retourner une valeur. SQL Server en propose plusieurs en natif, voici quelques exemples :

In [14]:
SELECT GETDATE() AS [Date], [Titre] FROM [Revue_REV]
GO

(0 rows affected)

Total execution time: 00:00:00.005

Date,Titre


In [15]:
SELECT
    COALESCE([Descriptif], [Contenu]) AS [Resultat],
    [Descriptif],
    [Contenu]
FROM [dbo].[Produit_PRD]
GO

(1 row affected)

Total execution time: 00:00:00.006

Resultat,Descriptif,Contenu
NULL,NULL,NULL


On va également pouvoir définir nos propres fonctions

In [16]:
CREATE OR ALTER FUNCTION [dbo].[CalculPromo](@Prix DECIMAL(5, 2))
RETURNS DECIMAL(5, 2)
AS
BEGIN
    DECLARE @Taux DECIMAL(5, 2);

    IF (@Prix > 50)
        SET @Taux = 0.65;
    ELSE
        SET @Taux = 0.75;

    RETURN(@Prix * @Taux)
END
GO

Commands completed successfully.

Total execution time: 00:00:00.006

In [17]:
SELECT [Prix], [dbo].[CalculPromo]([Prix]) AS [Promo] FROM [Produit_PRD]
GO

(1 row affected)

Total execution time: 00:00:00.012

Prix,Promo
"0,00","0,00"


## Triggers et curseurs

Avant d'entrer dans le vif du sujet, nous allons modifier notre table produits pour y ajouter deux colonnes :
- Une pour la date d'ajout de produit
- Une pour la date de mise à jour du produit

In [18]:
IF NOT EXISTS (
    SELECT [name] 
    FROM   [sys].[columns]
    WHERE  [object_id] = OBJECT_ID(N'[dbo].[Produit_PRD]') 
            AND [name] = 'Ajout'
)
    ALTER TABLE [dbo].[Produit_PRD]
    ADD [Ajout] DATETIME2 NOT NULL,
        [Modification] DATETIME2 NULL,
        DEFAULT GETDATE() FOR [Ajout]
GO

Commands completed successfully.

Total execution time: 00:00:00.011

Notre table de produit comporte désormais deux champs supplémentaire, une date avec une valeur par défaut (date d'ajout) et une date sans valeur par défaut (date de dernière modification).

Si j'ai envie de maintenir à jour la date de dernière modification de ma table sans devoir veiller à mettre la valeur correctement à chaque fois, je peux définir un trigger lors de l'update de ma (ou mes) ligne(s).

Pour ce faire, nous allons également utiliser un curseur pour itérer sur la (ou les) valeur(s) modifiée(s).

In [19]:
CREATE OR ALTER TRIGGER [dbo].[trig_UpdatePrdModification]
   ON  [dbo].[Produit_PRD] 
   AFTER UPDATE
AS 
BEGIN
	SET NOCOUNT ON;

    DECLARE @num INT
	DECLARE @PrdId INT

	SELECT @num = COUNT(*) from INSERTED

	IF @num > 1
	BEGIN
		DECLARE curs CURSOR FOR SELECT [Id] from INSERTED
		OPEN curs		
		FETCH NEXT FROM curs into @PrdId; 
		WHILE (@@FETCH_STATUS = 0)
		BEGIN
			UPDATE	[dbo].[Produit_PRD]
			SET		[Modification] = GETDATE()
			WHERE	[Id] = @PrdId

			FETCH NEXT FROM curs into @PrdId; 
		END

		CLOSE curs
		DEALLOCATE curs
	END
	ELSE
	BEGIN
		SELECT TOP 1 @PrdId = [Id] from INSERTED

		UPDATE  [dbo].[Produit_PRD]
        SET		[Modification] = GETDATE()
        WHERE	[Id] = @PrdId
	END
END

Commands completed successfully.

Total execution time: 00:00:00.009

## Procédures stockées

Les procédures stockées vont nous permettre de faire des suites d'actions sur notre base, avec un faible coût en temps.

Afin d'illustrer un cas d'utilisation, nous allons créer deux tables, une pour nos commandes, et une pour les produits qu'elles contiennent.

In [21]:
IF NOT EXISTS (
    SELECT  [name]
    FROM    [sysobjects]
    WHERE   [name]='Commande_CMD'
            AND [xtype]='U')
    CREATE TABLE [dbo].[Commande_CMD] (
        [Id] UNIQUEIDENTIFIER PRIMARY KEY DEFAULT NEWID(),
        [Date] DATETIME NOT NULL DEFAULT GETDATE()
    )
    GO

Commands completed successfully.

Total execution time: 00:00:00.011

In [23]:
IF NOT EXISTS (
    SELECT  [name]
    FROM    [sysobjects]
    WHERE   [name]='ProduitDeCommande_PDC'
            AND [xtype]='U')
    CREATE TABLE [dbo].[ProduitDeCommande_PDC] (
        [Id] UNIQUEIDENTIFIER PRIMARY KEY DEFAULT NEWID(),
        [CMD_Id] UNIQUEIDENTIFIER,
        [Nom] NVARCHAR(255) NOT NULL,
        [Prix] DECIMAL(5, 2) NOT NULL,
        [Descriptif] NVARCHAR(MAX),
        [Quantite] SMALLINT NOT NULL 
        CONSTRAINT fk_CMD_Id_PDC
            FOREIGN KEY ([CMD_Id]) REFERENCES [Commande_CMD]([Id])
    )
    GO

Commands completed successfully.

Total execution time: 00:00:00.020

Nous voulons faire la création de nos commande d'une traite. Dans un premier temps, nous allons faire des commandes avec un seul produit.

Notre procédure va ainsi créer notre nouvelle ligne de commande, puis attacher notre produit à cette commande grâce à l'ID de la commande, et ce, sans repasser par un client (.NET, Java, etc), entièrement côté SQL.

In [24]:
CREATE OR ALTER PROCEDURE [dbo].[prc_CreationCommande]
    @IdProduit BIGINT
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @CmdId UNIQUEIDENTIFIER = NEWID();

    INSERT INTO [dbo].[Commande_CMD] ([Id])
    VALUES (@CmdId)

    INSERT INTO [dbo].[ProduitDeCommande_PDC]
    ([CMD_Id], [Nom], [Prix], [Descriptif], [Quantite])
    SELECT @CmdId, [Nom], [Prix], [Descriptif], 1
    FROM [dbo].[Produit_PRD] AS [PRD] (NOLOCK)
    WHERE [PRD].[Id] = @IdProduit
END;

Commands completed successfully.

Total execution time: 00:00:00.023

On peut appeler les procédures à l'aide du mot-clé `EXEC` directement dans une fenêtre de requête, ou dans une autre procédure.

```sql
EXEC [dbo].[prc_CreationCommande] @IdProduit = 1
```

In [31]:
SELECT * FROM [ProduitDeCommande_PDC]

(1 row affected)

Total execution time: 00:00:00.008

Id,CMD_Id,Nom,Prix,Descriptif,Quantite
cead23b7-a2d9-4bfd-a56a-a383a3a91c6c,353234a8-1cc0-4105-8e35-bb0790e9bc2d,iPhone 11,"0,00",NULL,1


Si on veut passer plusieurs paramètre du même type, nous aurons tendance à utiliser des chaînes de caractères avec des sépérateurs.

In [32]:
CREATE OR ALTER PROCEDURE [dbo].[prc_CreationCommande]
    @IdsProduits NVARCHAR(MAX)
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @CmdId UNIQUEIDENTIFIER = NEWID();

    INSERT INTO [dbo].[Commande_CMD] ([Id])
    VALUES (@CmdId)

    INSERT INTO [dbo].[ProduitDeCommande_PDC]
    ([CMD_Id], [Nom], [Prix], [Descriptif], [Quantite])
    SELECT @CmdId, [Nom], [Prix], [Descriptif], 1
    FROM [dbo].[Produit_PRD] AS [PRD] (NOLOCK)
    WHERE [PRD].[Id] IN (
        SELECT CAST(value AS BIGINT)
        FROM STRING_SPLIT(@IdsProduits, '|')
    )
END;

Commands completed successfully.

Total execution time: 00:00:00.026

L'appel se ferait de la manière suivante :

```sql
EXEC [dbo].[prc_CreationCommande] @IdProduit = N'1|2|3|4'
```

__Attention !__

La méthodologie avec `NEWID()` ne fonctionne qu'avec le type `UNIQUEIDENTIFIER`. Si vos clé primaires sont des entier (`INT`, `BIGINT`, ...), vous devrez faire de la façon suivante :

```sql
DECLARE @NewId BIGINT

INSERT INTO [dbo].[MaTable] ([Champ1], [Champ2])
VALUES ('val1', 'val2')

SET @NewId = @@IDENTITY -- Contient la lé nouvellement générée
```

## WITH common_table_expression

Afin de passer plusieurs paramètres de façon simple à notre procédure, nous allons séparés nos champs de le façon suivante :

```
Id;Quentite|Id;Quantite
```

Pour pouvoir manipuler plus simplement nos données, nous allons pouvoir utiliser une expression `WITH common_table_expression` (ou `WITH CTE` pour faire plus court).

Ces epressions doivent forcément être précédées par un point-virgule (`;WITH ...`) afin de fonctionner. Elles expirent également immédiatement à la suite de la première instruction recontrée qui n'est pas une `WITH common_table_expression`.

In [45]:
CREATE OR ALTER PROCEDURE [dbo].[prc_CreationCommande]
    @InfosProduits NVARCHAR(MAX)
AS
BEGIN
    SET NOCOUNT ON;
     
    DECLARE @CmdId UNIQUEIDENTIFIER = NEWID()

    INSERT INTO [dbo].[Commande_CMD] ([Id])
    VALUES (@CmdId)

    ;WITH InfosProduits AS (
        SELECT  CAST(PARSENAME(REPLACE([value],';','.'),2) AS BIGINT) as [Id],
                CAST(PARSENAME(REPLACE([value],';','.'),1) AS SMALLINT) as [Quantite]
        FROM STRING_SPLIT(@InfosProduits, '|')
    )

    INSERT INTO [dbo].[ProduitDeCommande_PDC]
    ([CMD_Id], [Nom], [Prix], [Descriptif], [Quantite])
    SELECT @CmdId, [Nom], [Prix], [Descriptif], [IFP].[Quantite]
    FROM [dbo].[Produit_PRD] AS [PRD] (NOLOCK)
    INNER JOIN InfosProduits AS [IFP] ON [IFP].[Id] = [PRD].[Id];
END;

Commands completed successfully.

Total execution time: 00:00:00.027

Nous pouvons maintenant utiliser notre procédure de la manière suivante :

```sql
EXEC [dbo].[prc_CreationCommande] @InfosProduits = N'1;2|2;5'
```

Nous aurions également pu utiliser une variable de type `TABLE`, comme dans l'exemple ci-dessous :

In [51]:
CREATE OR ALTER PROCEDURE [dbo].[prc_CreationCommande]
    @InfosProduits NVARCHAR(MAX)
AS
BEGIN
    SET NOCOUNT ON;
     
    DECLARE @CmdId UNIQUEIDENTIFIER = NEWID()
    DECLARE @Table AS TABLE (
            [Id] BIGINT,
            [Quantite] SMALLINT
    )

    INSERT INTO @Table([Id], [Quantite])
    SELECT  CAST(PARSENAME(REPLACE([value],';','.'),2) AS BIGINT) as [Id],
                CAST(PARSENAME(REPLACE([value],';','.'),1) AS SMALLINT) as [Quantite]
        FROM STRING_SPLIT(@InfosProduits, '|')

    INSERT INTO [dbo].[Commande_CMD] ([Id])
    VALUES (@CmdId)

    INSERT INTO [dbo].[ProduitDeCommande_PDC]
    ([CMD_Id], [Nom], [Prix], [Descriptif], [Quantite])
    SELECT @CmdId, [Nom], [Prix], [Descriptif], [IFP].[Quantite]
    FROM [dbo].[Produit_PRD] AS [PRD] (NOLOCK)
    INNER JOIN @Table AS [IFP] ON [IFP].[Id] = [PRD].[Id];
END;

Commands completed successfully.

Total execution time: 00:00:00.018

Ces dernières ont quelques propriétés particulières :

- Elles sont créées en mémoire, contrairement aux tables temporaires
- Elles ne peuvent pas être utilisées dans des transactions, mais sont plus rapide que des tables temporaires
- Elles ne peuvent pas avoir d'index particuliers (seulement clé primaire et unique), contrairement aux tables temporaires
- Elles peuvent servir de paramètres aux procédures et aux fonctions, contrairement aux tables temporaires
- On ne peut pas modifier leur schéma, contrairement aux tables temporaires
- Elles ne peuvent être utilisées qu'entre deux routines (`GO`), contrairement aux tables temporaires